# 🚀 LangGraph QuickStart - 입문자를 위한 실전 튜토리얼

## 🎯 목표와 대상

이 노트북은 **LangGraph**를 처음 접하는 분을 위한 **실전 중심 튜토리얼** 입니다.

### 📋 이 튜토리얼에서 만들 기능

- **🧠 기억력**: 이전 대화를 유지하는 상태 관리
- **🔍 검색 연동**: 외부 검색 도구로 최신 정보 확보
- **🙋 인간 개입**: 승인 기반 Human-in-the-Loop
- **⪪ 상태 이력**: 체크포인트 기반 롤백 및 재실행

### 🗺️ 진행 로드맵

- **단계 1: 기본 챗봇 구축** – StateGraph, 메시지 흐름
- **단계 2: 도구 통합** – Tool 바인딩, 조건부 라우팅
- **단계 3: 메모리 추가** – 체크포인트, Thread ID
- **단계 4: Human-in-the-Loop** – interrupt, 승인 흐름
- **단계 5: 상태 커스터마이징** – 커스텀 State, Tool 업데이트
- **단계 6: 상태 이력 관리** – 이력 탐색, 롤백, 재실행

## 🛠️ 환경 설정

### 준비 항목

1. **🔑 API 키 설정** – OpenAI, Tavily, (선택) LangSmith
2. **📊 추적 설정** – LangSmith로 실행 추적 (선택)

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
# logging.langsmith("LangGraph-Tutorial")

---

# Part 1: 기본 챗봇 구축 🤖

## 🎯 목적

가장 단순한 형태의 **메시지 기반 챗봇**을 구성합니다.

### 구성 요소

- **🗺️ StateGraph**: 전체 흐름 정의
- **📝 State**: 메시지 저장 구조
- **🔨 Node**: 처리 함수 (LLM 호출)
- **🛤️ Edge**: 실행 경로 연결
- **⚙️ Compile/Invoke**: 실행 준비 및 호출

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# State 정의: 챗봇의 상태를 나타내는 타입
class State(TypedDict):
    """챗봇의 상태를 정의하는 타입
    
    messages: 대화 메시지 리스트
    - add_messages 함수를 통해 새 메시지가 추가됨 (덮어쓰기가 아닌 추가)
    """
    messages: Annotated[list, add_messages]


# StateGraph 생성
graph_builder = StateGraph(State)

print("✅ StateGraph 생성 완료!")
print("📌 State는 messages 키를 가지며, add_messages 리듀서를 사용합니다.")

### 🧠 LLM 선택 및 설정

실습에서는 **GPT-4o**를 사용합니다.

In [ ]:
# LLM 선택
from langchain_openai import ChatOpenAI

# OpenAI 모델 사용
llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 🔨 챗봇 노드 추가

대화 메시지를 입력받아 LLM에 전달하고, 응답 메시지를 상태에 추가합니다.

In [ ]:
def chatbot(state: State):
    """챗봇 노드 함수
    
    현재 상태의 메시지를 받아 LLM에 전달하고,
    응답을 새 메시지로 추가하여 반환합니다.
    """
    # LLM을 호출하여 응답 생성
    response = llm.invoke(state["messages"])
    
    # 응답을 메시지 리스트에 추가하여 반환
    return {"messages": [response]}


# 그래프에 노드 추가
# 첫 번째 인자: 노드의 고유 이름
# 두 번째 인자: 노드가 사용될 때 호출될 함수
graph_builder.add_node("chatbot", chatbot)

### 🚪 진입점과 종료점 설정

실행 경로를 명확히 정의합니다.

- **START**: 입력 수신
- **chatbot**: LLM 호출 및 응답 생성
- **END**: 결과 반환

In [ ]:
# 진입점: 그래프 실행이 시작되는 지점
graph_builder.add_edge(START, "chatbot")

# 종료점: 그래프 실행이 끝나는 지점
graph_builder.add_edge("chatbot", END)

print("✅ 진입점과 종료점 설정 완료!")
print("📌 실행 흐름: START → chatbot → END")

### ⚡ 그래프 컴파일

In [ ]:
# 그래프 컴파일
graph = graph_builder.compile()

print("✅ 그래프 컴파일 완료!")

### 👀 그래프 시각화

In [ ]:
from IPython.display import Image, display

try:
    # 그래프를 Mermaid 형식으로 시각화
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"시각화 실패: {e}")
    print("Graphviz가 설치되어 있지 않을 수 있습니다.")

### 🎉 챗봇 실행

In [ ]:
from langchain_core.messages import HumanMessage

# 질문 입력
user_input = "안녕하세요! LangGraph에 대해 알려주세요."

# 입력 메시지 준비
inputs = {
    "messages": [HumanMessage(content=user_input)]
}

# 그래프 실행
result = graph.invoke(inputs)

# 결과 출력
print(f"User: {user_input}\n")
print(f"Assistant: {result['messages'][-1].content}")

---

# Part 2: 도구(Tools) 추가 🔧

## 🎯 목적
실시간 정보가 필요한 요청에 대응하기 위해 외부 검색 도구를 통합합니다.

### 핵심 개념
- **Tool Binding**: LLM이 도구를 호출할 수 있도록 연결
- **Tool Node**: 외부 API 호출을 담당하는 노드
- **Conditional Edges**: 도구 사용 여부를 자동으로 분기

In [ ]:
# 간단한 도구 예시
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """두 숫자를 더합니다."""
    return a + b

@tool 
def multiply(a: int, b: int) -> int:
    """두 숫자를 곱합니다."""
    return a * b

# 도구 리스트
tools = [add, multiply]

print(f"✅ {len(tools)}개의 도구가 준비되었습니다: {[t.name for t in tools]}")

### 🗺️ 도구 사용 그래프 구성

기본 흐름에 도구 호출 경로를 추가합니다.

- 기존: 사용자 → chatbot → END
- 변경: 사용자 → chatbot ⇄ tools → chatbot → END

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# State 정의 (동일)
class State(TypedDict):
    messages: Annotated[list, add_messages]


# 새로운 그래프 빌더 생성
builder = StateGraph(State)

# LLM에 도구 바인딩 - LLM이 도구를 사용할 수 있도록 설정
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    """도구를 사용할 수 있는 챗봇 노드"""
    # 도구가 바인딩된 LLM 호출
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


# 노드 추가
builder.add_node("chatbot", chatbot)

# ToolNode 추가 - 도구를 실행하는 노드
tool_node = ToolNode(tools=tools)
builder.add_node("tools", tool_node)

### 🚦 조건부 라우팅(Conditional Edges)

요청에 따라 자동으로 경로를 선택합니다.

- **도구 호출 필요**: "tools" 로 이동
- **내부 처리 가능**: 종료

핵심: `tools_condition` 이 마지막 AI 메시지의 `tool_calls` 존재 여부를 확인합니다.

In [ ]:
# 조건부 엣지 추가
# tools_condition은 메시지에 tool_calls가 있으면 "tools"로,
# 없으면 END로 라우팅합니다
builder.add_conditional_edges(
    "chatbot",
    tools_condition,  # 사전 정의된 조건 함수 사용
)

# 도구 실행 후 다시 챗봇으로 돌아가기
builder.add_edge("tools", "chatbot")

# 시작점 설정
builder.add_edge(START, "chatbot")

# 그래프 컴파일
graph_with_tools = builder.compile()

print("✅ 도구가 포함된 그래프 구성 완료!")

### 👀 업그레이드 그래프 시각화

In [ ]:
try:
    display(Image(graph_with_tools.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"시각화 실패: {e}")

### 🚀 도구 사용 테스트

In [ ]:
# 도구를 사용하는 질문
test_input = "25 더하기 17은 얼마인가요? 그리고 그 결과에 3을 곱하면?"

inputs = {
    "messages": [HumanMessage(content=test_input)]
}

# 그래프 실행
print(f"User: {test_input}\n")
print("처리 과정:")
print("-" * 50)

# 스트리밍으로 실행 과정 확인
for event in graph_with_tools.stream(inputs):
    for node_name, output in event.items():
        if node_name == "chatbot":
            message = output["messages"][0]
            if hasattr(message, "tool_calls") and message.tool_calls:
                print(f"🤖 챗봇: 도구 호출 요청")
                for tool_call in message.tool_calls:
                    print(f"   - {tool_call['name']}({tool_call['args']})")
            else:
                print(f"🤖 챗봇: {message.content}")
        elif node_name == "tools":
            print(f"🔧 도구 실행 결과:")
            for msg in output["messages"]:
                print(f"   - {msg.content}")
        print("-" * 50)

## 📚 Part 2 요약

### 학습한 내용
1. **도구 정의**: `@tool` 데코레이터로 함수를 도구로 변환
2. **도구 바인딩**: `llm.bind_tools()`로 LLM에 도구 연결
3. **ToolNode**: 도구 실행을 담당하는 특수 노드
4. **조건부 라우팅**: `tools_condition`으로 자동 분기
5. **순환 구조**: 도구 → 챗봇 → 도구의 반복 가능

### 실무 활용 예시
- 실시간 데이터 조회 (날씨, 주가, 뉴스)
- 계산 및 분석 도구 연동
- 외부 API 호출 (검색, 번역, DB 조회)
- 멀티스텝 작업 자동화